In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import RandomSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda:0"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)

In [5]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = RandomSampler(res, train_index, test_index, null_mask)
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.697716 auc:0.4988
epoch:  20 loss:0.333574 auc:0.9195
epoch:  40 loss:0.308413 auc:0.9262
epoch:  60 loss:0.278838 auc:0.9330
epoch:  80 loss:0.253247 auc:0.9383
epoch: 100 loss:0.237162 auc:0.9411
epoch: 120 loss:0.227496 auc:0.9423
epoch: 140 loss:0.223137 auc:0.9425
epoch: 160 loss:0.217672 auc:0.9432
epoch: 180 loss:0.215757 auc:0.9431
epoch: 200 loss:0.212964 auc:0.9433
epoch: 220 loss:0.211394 auc:0.9432
epoch: 240 loss:0.211605 auc:0.9428
epoch: 260 loss:0.209013 auc:0.9431
epoch: 280 loss:0.209420 auc:0.9429
epoch: 300 loss:0.207486 auc:0.9429
epoch: 320 loss:0.208515 auc:0.9426
epoch: 340 loss:0.206418 auc:0.9429
epoch: 360 loss:0.206410 auc:0.9426
epoch: 380 loss:0.206599 auc:0.9424
epoch: 400 loss:0.205384 auc:0.9426
epoch: 420 loss:0.206734 auc:0.9423
epoch: 440 loss:0.204608 auc:0.9426
epoch: 460 loss:0.210107 auc:0.9398
epoch: 480 loss:0.204484 auc:0.9425
epoch: 500 loss:0.203555 auc:0.9425
epoch: 520 loss:0.207293 auc:0.9417
epoch: 540 loss:0.203486 auc

In [6]:
true_datas.to_csv("true_ctrp.csv")
predict_datas.to_csv("pred_ctrp.csv")